In [72]:
import pandas as pd
import numpy as np
import difflib
from thefuzz import fuzz, process

In [73]:
city_1 = pd.read_csv('uscities.csv')
city_2 = pd.read_csv('us_cities_3.csv')


In [74]:
city_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30844 entries, 0 to 30843
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   city          30844 non-null  object 
 1   city_ascii    30844 non-null  object 
 2   state_id      30844 non-null  object 
 3   state_name    30844 non-null  object 
 4   county_fips   30844 non-null  int64  
 5   county_name   30844 non-null  object 
 6   lat           30844 non-null  float64
 7   lng           30844 non-null  float64
 8   population    30844 non-null  int64  
 9   density       30844 non-null  float64
 10  source        30844 non-null  object 
 11  military      30844 non-null  bool   
 12  incorporated  30844 non-null  bool   
 13  timezone      30844 non-null  object 
 14  ranking       30844 non-null  int64  
 15  zips          30842 non-null  object 
 16  id            30844 non-null  int64  
dtypes: bool(2), float64(3), int64(4), object(8)
memory usage: 3.6+ MB


In [75]:
city_1 = city_1[['city_ascii', 'state_name', 'population', 'county_name', 'lat', 'lng']]
city_1 = city_1.rename(columns={
    "city_ascii":"city",
    "state_name":"state",
    "county_name":"county",
    "lat":"latitude",
    'lng':'longitude'
})

In [76]:
s = 'St. Louis'
a = s.replace(r'St\. ', 'Saint')
print(a)

St. Louis


In [77]:
city_1['city'] = city_1['city'].str.replace(r'St\. ', 'Saint ', regex=True)
city_1['county'] = city_1['county'].str.replace(r'St\. ', 'Saint ', regex=True)

In [78]:
city_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29880 entries, 0 to 29879
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          29880 non-null  int64  
 1   STATE_CODE  29880 non-null  object 
 2   STATE_NAME  29880 non-null  object 
 3   CITY        29880 non-null  object 
 4   COUNTY      29858 non-null  object 
 5   LATITUDE    29880 non-null  float64
 6   LONGITUDE   29880 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1.6+ MB


In [79]:
city_2 = city_2[['CITY', 'STATE_NAME', 'COUNTY', 'LATITUDE', 'LONGITUDE']]
city_2 = city_2.rename(columns={
    'CITY':'city',
    'STATE_NAME':'state',
    'COUNTY':'county',
    'LATITUDE':'latitude',
    'LONGITUDE':'longitude'
})


In [80]:
city_2['population'] = np.nan

In [81]:
all_cities = pd.concat([city_1, city_2], ignore_index=True)
all_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60724 entries, 0 to 60723
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        60724 non-null  object 
 1   state       60724 non-null  object 
 2   population  30844 non-null  float64
 3   county      60702 non-null  object 
 4   latitude    60724 non-null  float64
 5   longitude   60724 non-null  float64
dtypes: float64(3), object(3)
memory usage: 2.8+ MB


In [82]:
all_cities['city'] = all_cities['city'].str.lower().str.title()
all_cities['state'] = all_cities['state'].str.lower().str.title()
all_cities['county'] = all_cities['county'].str.lower().str.title()
all_cities = all_cities.drop_duplicates(subset=['city', 'state']).reset_index(drop=True)
all_cities

,city,state,population,county,latitude,longitude
0,New York,New York,18972871.0,Queens,40.694300,-73.924900
1,Los Angeles,California,12121244.0,Los Angeles,34.114100,-118.406800
2,Chicago,Illinois,8595181.0,Cook,41.837500,-87.686600
3,Miami,Florida,5711945.0,Miami-Dade,25.784000,-80.210100
4,Dallas,Texas,5668165.0,Dallas,32.793500,-96.766700
...,...,...,...,...,...,...
39445,Wapiti,Wyoming,NaN,Park,44.479670,-109.432629
39446,Weston,Wyoming,NaN,Campbell,44.835689,-105.373236
39447,Wolf,Wyoming,NaN,Sheridan,44.768228,-107.303429
39448,Wyarno,Wyoming,NaN,Sheridan,44.813333,-106.773333


In [83]:
all_cities = all_cities.reset_index(names='city_id')
all_cities = all_cities[~(all_cities['state'].isin(['Puerto Rico', 'District of Columbia', 'District Of Columbia']))]


In [84]:
print(all_cities['state'].nunique())

50


In [85]:
location = all_cities[['city_id', 'latitude', 'longitude']]
city = all_cities[['city_id', 'city', 'state', 'county', 'population']]

In [86]:
location.to_csv('cleaned/location.csv', index=False)
city.to_csv('cleaned/city.csv', index=False)